In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
from torchvision.transforms import ToTensor, Lambda, Normalize, CenterCrop, Resize
from torch import nn

from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive/MyDrive/

In [ ]:
ls

In [ ]:
!pip install import-ipynb
import import_ipynb

In [ ]:
from Custom_Read_Data import CustomReadData
from Custom_image_dataset import CustomImageDataset
from Train import Model_Training
import densenet 
import vgg_net
import resnet
import Inception_V3

In [ ]:
#CUDA for Pytorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
#device = 'cpu'
print(f'using {device} device')
#torch.backends.cudnn.benchmark = True

In [ ]:
model = resnet.ResNet182(img_channel=3, num_classes=3)
model.to(device)

In [ ]:
PATH = './res_net182.pth'

model.load_state_dict(torch.load(PATH))

In [ ]:
test_path = './test/'

files = glob(test_path + '*/*_image.jpg')

outfile = open('predictions_resnet_182.csv', 'w')

outfile.write("guid/image,label\n")

with torch.no_grad():
    for f in files:
        x = read_image(f)
        x = Resize((400, 800))(x)
        
        x = Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))(x.type(torch.float)) #Need to use the proper mean and std of the image planes
        x_shape = list(x.size())
        x = x.reshape((1, x_shape[0], x_shape[1], x_shape[2]))
        x = x.to(device)
        output = model(x)
        output = output.cpu().detach().numpy()
        out = np.argmax(output)
        print(out)
        
        f_split = f.split("/")     
        folder = f_split[2]    #check this line "\\" if error occurs and check the index values
        img_name = f_split[3].split("_")[0]
        file_name = folder + '/' + img_name
        
        outfile.write(f"{file_name},{out}\n")

outfile.close()

In [ ]:
#Optional method
#Load model using checkpoint

loaded_checkpoint = torch.load("./checkpoints_resnet_182.pth")
epoch = loaded_checkpoint["epoch"]
print(epoch)

model = resnet.ResNet182(img_channel=3, num_classes=3)
model.to(device)

learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

model.load_state_dict(loaded_checkpoint["model_state"])
optimizer.load_state_dict(loaded_checkpoint["optim_state"])